# 4) Intro to Shapely and Fiona

In [1]:
import csv
import fiona
from shapely.geometry import Point, Polygon, mapping
from collections import OrderedDict

## Reading in Data

In [8]:
with open('data/cities_population.csv') as reader:
    text = csv.reader(reader)
    for row in text:
        print(row)

['Country', 'City', 'Population', 'Latitude', 'Longitude']
['cn', 'shanghai', '14608512', '31.045556', '121.399722']
['br', 'sao paulo', '10021437', '-23.473293', '-46.665803']
['cd', 'kinshasa', '7787832', '-4.3', '15.3']
['eg', 'cairo', '7734602', '30.05', '31.25']
['cn', 'peking', '7480601', '39.928889', '116.388333']
['gb', 'london', '7421228', '51.514125', '-0.093689']
['co', 'bogota', '7102602', '4.649178', '-74.062827']
['bd', 'dhaka', '6493177', '23.7230556', '90.4086111']
['br', 'rio de janeiro', '6023742', '-22.9', '-43.233333']
['cl', 'santiago', '4837248', '-33.45', '-70.666667']
['ca', 'toronto', '4612187', '43.666667', '-79.416667']
['au', 'sydney', '4394585', '-33.861481', '151.205475']
['cn', 'wuhan', '4184206', '30.580125', '114.273405']
['cn', 'chongqing', '3967028', '29.562778', '106.552778']
['eg', ' alexandria', '3811512', '31.1980556', '29.9191667']
['eg', 'cairo', '7734602', '30.05', '31.25']


In [9]:
with open('data/cities_population.csv') as reader:
    text = csv.DictReader(reader)
    for row in text:
        print(row)

{'Country': 'cn', 'City': 'shanghai', 'Population': '14608512', 'Latitude': '31.045556', 'Longitude': '121.399722'}
{'Country': 'br', 'City': 'sao paulo', 'Population': '10021437', 'Latitude': '-23.473293', 'Longitude': '-46.665803'}
{'Country': 'cd', 'City': 'kinshasa', 'Population': '7787832', 'Latitude': '-4.3', 'Longitude': '15.3'}
{'Country': 'eg', 'City': 'cairo', 'Population': '7734602', 'Latitude': '30.05', 'Longitude': '31.25'}
{'Country': 'cn', 'City': 'peking', 'Population': '7480601', 'Latitude': '39.928889', 'Longitude': '116.388333'}
{'Country': 'gb', 'City': 'london', 'Population': '7421228', 'Latitude': '51.514125', 'Longitude': '-0.093689'}
{'Country': 'co', 'City': 'bogota', 'Population': '7102602', 'Latitude': '4.649178', 'Longitude': '-74.062827'}
{'Country': 'bd', 'City': 'dhaka', 'Population': '6493177', 'Latitude': '23.7230556', 'Longitude': '90.4086111'}
{'Country': 'br', 'City': 'rio de janeiro', 'Population': '6023742', 'Latitude': '-22.9', 'Longitude': '-43.2

In [12]:
with open('data/cities_population.csv') as reader:
    text = csv.DictReader(reader)
    for row in text:
        print(Point(float(row['Latitude']), float(row['Longitude'])))

POINT (31.045556 121.399722)
POINT (-23.473293 -46.665803)
POINT (-4.3 15.3)
POINT (30.05 31.25)
POINT (39.928889 116.388333)
POINT (51.514125 -0.09368899999999999)
POINT (4.649178 -74.062827)
POINT (23.7230556 90.4086111)
POINT (-22.9 -43.233333)
POINT (-33.45 -70.666667)
POINT (43.666667 -79.416667)
POINT (-33.861481 151.205475)
POINT (30.580125 114.273405)
POINT (29.562778 106.552778)
POINT (31.1980556 29.9191667)
POINT (30.05 31.25)


In [13]:
with open('data/cities_population.csv') as reader:
    text = csv.DictReader(reader)
    for row in text:
        print(row['City'], row['Population'])

shanghai 14608512
sao paulo 10021437
kinshasa 7787832
cairo 7734602
peking 7480601
london 7421228
bogota 7102602
dhaka 6493177
rio de janeiro 6023742
santiago 4837248
toronto 4612187
sydney 4394585
wuhan 4184206
chongqing 3967028
 alexandria 3811512
cairo 7734602


## Writing Points to Shapefile and GeoJSON

Shapefile schema previously looked like this (so we can follow the same conventions):

      {'properties': 

        OrderedDict(
            [('pop_est', 'int:18'),
              ('continent', 'str:80'),
              ('name', 'str:80'),
              ('iso_a3', 'str:80'),
              ('gdp_md_est', 'float:24.15')]
              ),
              
             'geometry': 'Polygon'}

In [14]:
# prepare schema

schema = {
    'geometry':'Point',
    'properties': OrderedDict([('Country', 'str'),
              ('City', 'str'),
              ('Population', 'int'),
              ('Latitude', 'float'),
              ('Longitude', 'float')])
}

schema

{'geometry': 'Point',
 'properties': OrderedDict([('Country', 'str'),
              ('City', 'str'),
              ('Population', 'int'),
              ('Latitude', 'float'),
              ('Longitude', 'float')])}

In [22]:
# write the Shapefile

with fiona.open('output/cities_ge.shp', 'w', driver='ESRI Shapefile', crs={'init':'epsg:4326'}, schema=schema) as file:
    reader = csv.DictReader(open('data/cities_population.csv'))
    for row in reader:
        point = Point(float(row['Latitude']), float(row['Longitude']))
        properties = {
            'Country':row['Country'],
            'City':row['City'],
            'Population':row['Population'],
            'Latitude':row['Latitude'],
            'Longitude':row['Longitude']
        }
        
        file.write({"geometry":mapping(point), # maps latitude and longitude into point gemoetry
                 "properties":properties}) 

In [23]:
# write the GeoJSON

with fiona.open('output/cities_ge.geojson', 'w', driver='GeoJSON', crs={'init':'epsg:4326'}, schema=schema) as file:
    reader = csv.DictReader(open('data/cities_population.csv'))
    for row in reader:
        point = Point(float(row['Latitude']), float(row['Longitude']))
        properties = {
            'Country':row['Country'],
            'City':row['City'],
            'Population':row['Population'],
            'Latitude':row['Latitude'],
            'Longitude':row['Longitude']
        }
        
        file.write({"geometry":mapping(point), # maps latitude and longitude into point gemoetry
                 "properties":properties}) 

## Writing Polygons

In [24]:
fiji_coordinates = [(180.0, -16.067132663642447),
     (180.0, -16.555216566639196),
     (179.36414266196414, -16.801354076946883),
     (178.72505936299711, -17.01204167436804),
     (178.59683859511713, -16.639150000000004),
     (179.0966093629971, -16.433984277547403),
     (179.4135093629971, -16.379054277547404),
     (180.0, -16.067132663642447), (178.12557, -17.50481),
     (178.3736, -17.33992), (178.71806, -17.62846),
     (178.55271, -18.15059), (177.93266000000003, -18.28799),
     (177.38146, -18.16432), (177.28504, -17.72465),
     (177.67087, -17.381140000000002), (178.12557, -17.50481),
     (-179.79332010904864, -16.020882256741224), (-179.9173693847653, -16.501783135649397),
     (-180.0, -16.555216566639196), (-180.0, -16.067132663642447),
     (-179.79332010904864, -16.020882256741224)]

In [28]:
fiji_schema = {
    'geometry':'Polygon',
    'properties': {'name':'str'}
}

fiji_schema

{'geometry': 'Polygon', 'properties': {'name': 'str'}}

In [29]:
# write the GeoJSON

with fiona.open('output/fiji.geojson', 'w', driver='GeoJSON', crs={'init':'epsg:4326'}, schema=fiji_schema) as file:
    file.write({
        "geometry":mapping(Polygon(fiji_coordinates)),
        "properties":{"name":"Fiji"}
    })